<h1><center>Laboratorio 8: ¿Superhéroe o Villano? 🦸</center></h1>

<center><strong>MDS7202: Laboratorio de Programación Científica para Ciencia de Datos</strong></center>

### Cuerpo Docente:

- Profesor: Matías Rojas y Mauricio Araneda
- Auxiliar: Ignacio Meza D.
- Ayudante: Rodrigo Guerra

### Equipo: SUPER IMPORTANTE - notebooks sin nombre no serán revisados

- Nombre de alumno 1: Matías Villa


### **Link de repositorio de GitHub:** `https://github.com/matiasjvd/MDS7202-Laboratorios.git`

## Temas a tratar

- Codificación de texto usando Bag of Words.
- Búsqueda del modelo óptimo de clasificación usando `GridSearch`
- Uso de pipelines.

## Reglas:

- **Grupos de 2 personas**
- **Ausentes** deberán realizar la actividad solos. 
- Cualquier duda fuera del horario de clases al foro. Mensajes al equipo docente serán respondidos por este medio.
- Prohibidas las copias. 
- Pueden usar cualquer matrial del curso que estimen conveniente.

### Objetivos principales del laboratorio

- Obtener caracteristicas a partir de texto usando `CountVectorizer`.
- Fijar un pipeline con un modelo base que luego se irá optimizando.
- Comprender como realizar una búsqueda de grilla sobre un conjunto de clasificadores e hiperparámetros usando `GridSearch`.

El laboratorio deberá ser desarrollado sin el uso indiscriminado de iteradores nativos de python (aka "for", "while"). La idea es que aprendan a exprimir al máximo las funciones optimizadas que nos entrega `pandas`, las cuales vale mencionar, son bastante más eficientes que los iteradores nativos sobre DataFrames.

#Importamos librerias utiles 😸

In [1]:
# Librería Core del lab.
import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline


from sklearn.model_selection import train_test_split 

# Pre-procesamiento
from sklearn.feature_selection import SelectPercentile, f_classif
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_extraction.text import CountVectorizer

# Clasifación
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

# Metricas de evaluación
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import cohen_kappa_score

# Librería para plotear
!pip install --upgrade plotly
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

# Proyecciones en baja dimensionalidad: UMAP
!pip install umap-learn

# Librería para NLP
!pip install nltk
import nltk
from nltk.corpus import stopwords
from nltk import word_tokenize  
from nltk.stem import PorterStemmer
nltk.download('stopwords')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

# 1. ¿Quien es Bat Cow?

<p align="center">
  <img src="https://i.imgur.com/D9f1RHy.jpg" width="350">
</p>

En vez de estar desarrollando las evaluaciones correspondientes a su curso, su profesor de catedra y su auxiliar discuten acerca la alineación (héroe o villano) del personaje de ficción Bat-Cow. 

El cuerpo docente, no logra ponerse de acuerdo si el personaje es bueno, neutral o malo: el auxiliar plantea que Bat-cow posee una siniestra mirada, intrigante pero común característica de los personajes malvados. 
Por otra parte, extendiendo las ideas de Rousseau, el profesor plantea que tal como los humanos no nacen malos, no existe motivo por el cual una vaca con superpoderes deba serlo.

Sin embargo, ambos concuerdan que es difícil estimar la alineación solo usando los atributos físicos, por lo que creen el análisis debe ser complementado aún más antes de comunicarle los resultados a su estudiantado. Buscando más información, ambos sujetos se percatan de la existencia de un excelente antecedente para estimar la alineación: la historia personal de cada superhéroe o villano.

Es por esto le solicitan que construya y optimice un clasificador basado en texto el cual analice la alineación de cada personaje basado en su historia personal.

Para este laboratorio deben trabajar con los datos `df_comics.csv` y `comics_no_label.csv` subidos a u-cursos. El primero es un conjunto de datos que les servirá para entrenar un modelo de clasificación, mientras que el segundo es un dataset con personajes de ficción no etiquetados a predecir (sí, aquí está la misteriosa Batcow).

Para comenzar cargue los dataset señalados y visualice a través de un head los atributos que poseen cada uno de los dataset.


In [2]:
# Si usted está utilizando Colabolatory le puede ser útil este código para cargar los archivos.
try:
    from google.colab import drive
    drive.mount("/content/drive")
    path = '/content/drive/MyDrive/Laboratorio_8'
except: 
    print('Ignorando conexión drive-colab')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:

df_comics = pd.read_csv('/content/drive/MyDrive/Laboratorio_8/df_comics.csv')
df_comics_no_label = pd.read_csv('/content/drive/MyDrive/Laboratorio_8/comics_no_label.csv')
df_comics = df_comics.dropna(subset=['history_text']) # eliminar ejemplos sin historia


In [4]:
# queda a labor de su equipo hacer el análisis exploratorio
df_comics.head()

,Unnamed: 0,name,real_name,full_name,overall_score,history_text,powers_text,intelligence_score,strength_score,speed_score,...,has_flight,has_accelerated_healing,has_weapons_master,has_intelligence,has_reflexes,has_super_speed,has_durability,has_stamina,has_agility,has_super_strength
0,0,3-D Man,"Delroy Garrett, Jr.","Delroy Garrett, Jr.",6,"Delroy Garrett, Jr. grew up to become a track ...",NaN,85,30,60,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
1,2,A-Bomb,Richard Milhouse Jones,Richard Milhouse Jones,20,"Richard ""Rick"" Jones was orphaned at a young ...","On rare occasions, and through unusual circu...",80,100,80,...,0.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0
2,3,Aa,Aa,NaN,12,Aa is one of the more passive members of the P...,NaN,80,50,55,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4,Aaron Cash,Aaron Cash,Aaron Cash,5,Aaron Cash is the head of security at Arkham A...,NaN,80,10,25,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,Aayla Secura,Aayla Secura,NaN,8,ayla Secura was a Rutian Twi'lek Jedi Knight (...,NaN,90,40,45,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [5]:
df_comics_no_label.head()

,Unnamed: 0,name,real_name,full_name,overall_score,history_text,powers_text,intelligence_score,strength_score,speed_score,...,has_flight,has_accelerated_healing,has_weapons_master,has_intelligence,has_reflexes,has_super_speed,has_durability,has_stamina,has_agility,has_super_strength
0,1,514A (Gotham),Bruce Wayne,NaN,10,He was one of the many prisoners of Indian Hil...,NaN,100,20,30,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
1,14,Ace Morgan,Kyle Morgan,NaN,7,NaN,"Aviation: Ace is an extremely skilled pilot, ...",85,20,65,...,0.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0
2,17,A'dal,NaN,NaN,7,"As with most of the naaru, little is known of ...",NaN,85,30,70,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,27,Agent Zero (FOX),David North,NaN,6,"During mid-late 1973, Zero was a member of Tea...",Zero can absorb kinetic energy to further incr...,90,10,25,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
4,31,Ajax (FOX),Francis,Francis Freeman,7,Ajax (born Francis Freeman) was a human who ga...,Ajax has claimed that the procedure to obtain ...,85,25,45,...,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0


In [6]:
#Ahora veamos la distribución de las variables numéricas
df_comics.describe()

,Unnamed: 0,intelligence_score,strength_score,speed_score,durability_score,power_score,combat_score,has_electrokinesis,has_energy_constructs,has_mind_control_resistance,...,has_flight,has_accelerated_healing,has_weapons_master,has_intelligence,has_reflexes,has_super_speed,has_durability,has_stamina,has_agility,has_super_strength
count,1285.000000,1285.000000,1285.000000,1285.000000,1285.000000,1285.000000,1285.000000,1228.000000,1228.000000,1228.000000,...,1228.000000,1228.000000,1228.000000,1228.000000,1228.000000,1228.000000,1228.000000,1228.000000,1228.000000,1228.000000
mean,727.744747,78.665370,38.832685,45.898833,55.610895,66.556420,67.805447,0.089577,0.074919,0.074104,...,0.265472,0.307003,0.403909,0.429153,0.437296,0.387622,0.504886,0.540717,0.566775,0.522801
std,415.297480,25.074439,34.220845,29.176614,31.683003,33.329484,29.598315,0.285690,0.263367,0.262047,...,0.441764,0.461439,0.490880,0.495157,0.496255,0.487406,0.500180,0.498542,0.495723,0.499683
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,372.000000,75.000000,10.000000,25.000000,30.000000,40.000000,55.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,728.000000,85.000000,30.000000,40.000000,55.000000,70.000000,75.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000
75%,1084.000000,95.000000,65.000000,65.000000,85.000000,100.000000,90.000000,0.000000,0.000000,0.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
max,1449.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [7]:
df_comics_no_label.describe()

,Unnamed: 0,intelligence_score,strength_score,speed_score,durability_score,power_score,combat_score,alignment,has_electrokinesis,has_energy_constructs,...,has_flight,has_accelerated_healing,has_weapons_master,has_intelligence,has_reflexes,has_super_speed,has_durability,has_stamina,has_agility,has_super_strength
count,84.000000,84.000000,84.000000,84.000000,84.000000,84.000000,84.000000,0.0,78.000000,78.000000,...,78.000000,78.000000,78.000000,78.000000,78.000000,78.000000,78.000000,78.000000,78.000000,78.000000
mean,613.369048,82.321429,40.476190,47.380952,57.619048,64.523810,69.226190,NaN,0.102564,0.038462,...,0.179487,0.230769,0.448718,0.320513,0.269231,0.243590,0.500000,0.346154,0.397436,0.500000
std,463.554958,23.624146,31.168209,25.239530,28.469770,33.470172,27.935964,NaN,0.305352,0.193552,...,0.386244,0.424052,0.500582,0.469694,0.446431,0.432026,0.503236,0.478822,0.492535,0.503236
min,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,193.250000,80.000000,13.750000,30.000000,40.000000,40.000000,60.000000,NaN,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,537.500000,90.000000,30.000000,50.000000,60.000000,70.000000,75.000000,NaN,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.500000,0.000000,0.000000,0.500000
75%,1075.250000,95.000000,61.250000,60.000000,80.000000,100.000000,90.000000,NaN,0.000000,0.000000,...,0.000000,0.000000,1.000000,1.000000,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000
max,1440.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,NaN,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [8]:
#Veamos la cantidad de valores nulos por columna
df_comics.isna().sum()

Unnamed: 0              0
name                    0
real_name             121
full_name             390
overall_score           0
                     ... 
has_super_speed        57
has_durability         57
has_stamina            57
has_agility            57
has_super_strength     57
Length: 82, dtype: int64

In [9]:
df_comics_no_label.isna().sum()

Unnamed: 0             0
name                   2
real_name             18
full_name             71
overall_score          0
                      ..
has_super_speed        6
has_durability         6
has_stamina            6
has_agility            6
has_super_strength     6
Length: 82, dtype: int64

In [10]:
#Ahora veamos la distribución de las variables categóricas
df_comics.describe(include=['O'])

,name,real_name,full_name,overall_score,history_text,powers_text,superpowers,alter_egos,aliases,place_of_birth,...,teams,relatives,gender,type_race,height,weight,eye_color,hair_color,skin_color,img
count,1285,1164,895,1285,1285,961,1285,1285,1285,734,...,1285,829,1191,942,1285,1285,1091,1103,154,1215
unique,1285,971,794,87,1274,939,1166,463,860,422,...,390,775,2,63,116,242,25,30,15,1215
top,3-D Man,Unknown,Unknown,6,id='history',No known powers.,[],[],[''],Earth-1,...,[],None,Male,Human,-,-,Blue,Black,Green,/pictures2/portraits/11/050/10038.jpg?v=156096...
freq,1,20,15,151,3,6,57,823,398,31,...,779,23,881,492,276,329,341,310,28,1


In [11]:
df_comics_no_label.describe(include=['O'])

,name,real_name,full_name,overall_score,history_text,powers_text,superpowers,alter_egos,aliases,place_of_birth,...,teams,relatives,gender,type_race,height,weight,eye_color,hair_color,skin_color,img
count,82,66,13,84,76,43,84,84,84,20,...,84,26,39,37,82,82,28,29,5,72
unique,81,63,12,24,75,43,77,30,32,19,...,10,26,2,15,16,17,8,8,4,71
top,Batcow,Bruce Wayne,None,7,Bat-Cow was originally a cow that was found by...,"Aviation: Ace is an extremely skilled pilot, ...",[],[],[''],Gotham City,...,[],Bruce Wayne (genetic template),Male,Human,-,-,Brown,Black,Blue,/pictures2/portraits/11/050/13425.jpg?v=157425...
freq,2,2,2,15,2,1,6,54,52,2,...,75,1,29,13,59,63,8,9,2,2


## 1.1 Obtención de Features y Bag of Words

<p align="center">
  <img src="https://media0.giphy.com/media/eIUpSyzwGp0YhAMTKr/200.gif" width="300">
</p>

Primero que todo, deben obtener un vector de características del atributo `history_text`, utilizando `Bag of Words`. En este atributo se presenta una breve descripción de la historia de cada uno de los personajes de ficción presentes en el dataset. 

Pero... antes de empezar, ¿Que es `Bag of Words`?...

`Bag of Words` es un modelo de conteo utilizado en Procesamiento de Lenguaje Natural (NLP) que tiene como objetivo generar una representación vectorial (vector de características en nuestro cas) para cada documento a través del conteo de las palabras que contienen. 

La siguiente figura muestra un ejemplo de `Bag of Words` en acción:

<p align="center">
  <img src="https://user.oc-static.com/upload/2020/10/23/16034397439042_surfin%20bird%20bow.png" width="500">
</p>

Como pueden ver, el modelo de `Bag of Words` no resulta tan complicado, ¿pero cómo lo aplicamos en python?. 

Como podrán darse cuenta del ejemplo anterior, para facilitar el conteo será necesario transformar cada uno de los documentos en vectores, donde cada una de las posiciones posee un carácter. Este proceso es conocido como **tokenización** y lo podemos realizar de la siguiente forma:

In [12]:
import nltk
nltk.download('punkt')
docs = ['The teacher rocks like a good rock & roll',
             'the rock is the best actor in the world']

from nltk.tokenize import word_tokenize

docs_tokenizados = [word_tokenize(doc)  for doc in docs]
docs_tokenizados

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


[['The', 'teacher', 'rocks', 'like', 'a', 'good', 'rock', '&', 'roll'],
 ['the', 'rock', 'is', 'the', 'best', 'actor', 'in', 'the', 'world']]

Podemos mejorar un poco más el proceso de tokenización agregando 

- Stemming:  Definimos Stemming como un algoritmo basado en reglas que transforma las palabras a una forma general. Un ejemplo de stemming, es el siguiente:
- Eliminación de Stopwords: Eliminación de palabras muy frecuentes que entorpecen la clasificación (por ejemplo, el, la los, la, etc...)

<p align="center">
  <img src="https://devopedia.org/images/article/218/8583.1569386710.png" width="300">
</p>


In [13]:
# Definimos algunas stopword que queremos que sean eliminadas
import nltk
nltk.download('stopwords')

from nltk.corpus import stopwords

stop_words = stopwords.words('spanish')

# Definimos un tokenizador con Stemming
class StemmerTokenizer:
    def __init__(self):
        self.ps = PorterStemmer()
    def __call__(self, doc):
        doc_tok = word_tokenize(doc)
        doc_tok = [t for t in doc_tok if t not in stop_words]
        return [self.ps.stem(t) for t in doc_tok]

# Inicializamos tokenizador
tokenizador = StemmerTokenizer()

# Creamos algunos documentos
docs = ['The teacher rocks like a good rock & roll',
        'the rock is the best actor in the world',
        'New York is a beautiful city']

# Obtenemos el token del primer documento
[tokenizador(doc) for doc in docs]

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


[['the', 'teacher', 'rock', 'like', 'good', 'rock', '&', 'roll'],
 ['the', 'rock', 'is', 'the', 'best', 'actor', 'in', 'the', 'world'],
 ['new', 'york', 'is', 'beauti', 'citi']]

In [14]:
# Comparación con el caso anterior
docs_tokenizados = [word_tokenize(doc) for doc in docs]
docs_tokenizados

[['The', 'teacher', 'rocks', 'like', 'a', 'good', 'rock', '&', 'roll'],
 ['the', 'rock', 'is', 'the', 'best', 'actor', 'in', 'the', 'world'],
 ['New', 'York', 'is', 'a', 'beautiful', 'city']]

#### Al Estilo Scikit

Scikit implementa `bag of words` a través de la clase `CountVectorizer()` la cual contiene muchas opciones para mejorar la tokenización.

In [15]:
bow = CountVectorizer(tokenizer= StemmerTokenizer())
df = bow.fit_transform(docs)

pd.DataFrame(df.toarray(), columns=bow.get_feature_names_out())

,&,actor,beauti,best,citi,good,in,is,like,new,rock,roll,teacher,the,world,york
0,1,0,0,0,0,1,0,0,1,0,2,1,1,1,0,0
1,0,1,0,1,0,0,1,1,0,0,1,0,0,3,1,0
2,0,0,1,0,1,0,0,1,0,1,0,0,0,0,0,1


Una de las cosas más interesantes que provee son el use de n-gramas, los cuales, en palabras simples, son conjuntos de n-palabras que se concatenan entre si y que se consideran como tokens separados. 

Pensemos en `Nueva York`. Cuando se tokeniza Nueva York, se generan dos tokens independientes que a simple vista no tienen relación: `Nueva` `York`.
Al usar n-gramas (en un rango min=1,max=2) , generamos tanto `Nueva` y `York` como también `Nueva York` como un token independiente.

In [16]:
bow = CountVectorizer(tokenizer= StemmerTokenizer(), ngram_range=(1,2))
df = bow.fit_transform(docs)

pd.DataFrame(df.toarray(), columns=bow.get_feature_names_out())

,&,& roll,actor,actor in,beauti,beauti citi,best,best actor,citi,good,...,teacher,teacher rock,the,the best,the rock,the teacher,the world,world,york,york is
0,1,1,0,0,0,0,0,0,0,1,...,1,1,1,0,0,1,0,0,0,0
1,0,0,1,1,0,0,1,1,0,0,...,0,0,3,1,1,0,1,1,0,0
2,0,0,0,0,1,1,0,0,1,0,...,0,0,0,0,0,0,0,0,1,1


De los resultados, podemos ver que generamos vectores de conteo para cada una de las palabras que conforman el corpus.  Un punto extra que se agrega en esta obtención de frecuencias son los bigramas, que básicamente son el conjunto de palabras de tamaño de aparecen juntas en el texto.

## Codificando los Super{heroes, villanos}  [0.5 Puntos]

<p align="center">
  <img src="https://c.tenor.com/LkQzw7k5DV4AAAAd/anime-hacking.gif" width="300">
</p>

Conociendo ahora que es el proceso de `bag of words`, aplique este modelo de obtención de caracteristicas de la siguiente forma en un pipeline:

- Utilice el tokenizador entregado.
- Obtenga caracteristicas de los unigramas y bigramas del texto (tal como el ejemplo).

```python
bog = CountVectorizer(tokenizer= StemmerTokenizer(),`
                      ngram_range=(1,2) # Este punto es opcional y es para generar bigramas
                      )
```

Finalmente, aplique `MinMaxScaler()` sobre `atributos_de_interes` y concatene el valor obtenido con el matriz de caracteristicas obtenidas con bag of words.

```python
atributos_de_interes = ['intelligence_score', 'strength_score', 'speed_score', 'durability_score', 'power_score', 'combat_score']
```

No es necesario que obtenga un dataframe en concreto con las características solicitadas. Se le recomienda generar un `ColumnTransformer()` para aplicar las transformaciones solicitadas en un pipeline.

**To-Do:**
- [ ] Obtener a traves de Bag of Words (`CountVectorizer`) caracteristicas del resumen de historia de cada personaje.
- [ ] Aplicar `MinMaxScaler` sobre los atributos de interes.

**Respuesta:**

In [17]:
from sklearn.preprocessing import MinMaxScaler

bog = CountVectorizer(tokenizer= StemmerTokenizer(),
                      ngram_range=(1,2) 
                      )


atributos_de_interes = ['intelligence_score', 'strength_score', 'speed_score', 'durability_score', 'power_score', 'combat_score']

scaler = MinMaxScaler()

preprocessor = ColumnTransformer(transformers = [('Texto', bog, 'history_text'), ('Numeric', scaler, atributos_de_interes ),])


## 1.2 Diseño de Baseline y  Primer Entrenamiento  [1 Puntos]


<p align="center">
  <img src="https://pa1.narvii.com/6374/9eaec1b7bf9157334151452a669516f9a78b954c_hq.gif" width="300">
</p>



Genere un Pipeline con las caracteristicas solicitadas en la sección 1.1, un selector de mejores features `SelectPercentile` con métrica `f_classif` y percentile=90 y un clasificador `MultinomialNB()` por defecto.

Luego, separe el conjunto de datos en un conjunto de entrenamiento y prueba, donde las etiquetas estará dado por el atributo `alignment`. 

Entrene el modelo y reporte el desempeño con un `classification_report`. ¿ Nos recomendaría predecir la alineación de BatCow con este clasificador?.

Finalmente, compare el modelo entrenado con un modelo Dummy estratificado y responda: ¿El clasificador entrenado es mejor que el dummy que entrega respuestas al azar?

**To-do:**
- [X] Realizar un pipeline con las caracteristicas solicitadas en 1.1, ejecutar holdout y aplicar un clasificador `MultinomialNB()`.
- [X] Entrenar el pipeline, calcular el `classification_report` asociado y comentar los resultados.
- [X] Entrenar un `DummyClassifier` con estrategia `statified`, calcular el `classification_report` asociado y comentar que implican los scores obtenidos en comparación con los resultados del baseline.

**Respuesta:**

In [18]:
#Primero realizamos el Pipeline con las caracteristicas solicitadas
model = Pipeline(steps=[('Preprocessing', preprocessor),
                       ('Selection', SelectPercentile(f_classif, percentile=90)),
                       ('Clasificador', MultinomialNB())])

#Ahora dividimos los datos en entrenamiento y test para poder entrenar el pipeline anterior

atributos = ['intelligence_score', 'strength_score', 'speed_score', 'durability_score', 'power_score', 'combat_score', 'history_text']
X = df_comics[atributos]
y = df_comics["alignment"]


X_train, X_test, y_train, y_test = train_test_split(X, y,shuffle=True, stratify = y,test_size=0.20, random_state=42)

#Ahora entrenamos el modelo
model.fit(X_train, y_train)

#Vemos como predice nuestro modelo
y_pred = model.predict(X_test)

#Comparamos con classification_report
print('Classification report')
print(classification_report(y_test, y_pred))



Classification report
              precision    recall  f1-score   support

         Bad       0.69      0.10      0.18        86
        Good       0.60      0.99      0.74       148
     Neutral       0.00      0.00      0.00        23

    accuracy                           0.60       257
   macro avg       0.43      0.36      0.31       257
weighted avg       0.58      0.60      0.49       257



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [19]:
#Dado que posiblemente exista un desbalance en los datos probamos con la metrica de Balanced Accuracy
from sklearn.metrics import balanced_accuracy_score
print('Balance Accuracy Score modelo MultinomialNB : ', balanced_accuracy_score(y_test, y_pred))

Balance Accuracy Score modelo MultinomialNB :  0.3637125497590614


In [20]:
#Finalmente entrenamos el Dummy Classifier con estrategia statified
from sklearn.dummy import DummyClassifier

model_dummy= Pipeline(
    steps=[
        ("Preprocessing",  preprocessor),
        ("Tree", DummyClassifier(strategy="stratified")),
    ]
)
#Entrenamos el modelo
model_dummy.fit(X_train, y_train)

#Vemos como predice nuestro modelo dummy
y_pred = model_dummy.predict(X_test)

#Comparamos con classification_report
print('Classification report')
print(classification_report(y_test, y_pred))




Classification report
              precision    recall  f1-score   support

         Bad       0.32      0.31      0.32        86
        Good       0.55      0.57      0.56       148
     Neutral       0.05      0.04      0.05        23

    accuracy                           0.44       257
   macro avg       0.31      0.31      0.31       257
weighted avg       0.43      0.44      0.43       257



In [21]:
#Dado que posiblemente exista un desbalance en los datos probamos con la metrica de Balanced Accuracy
from sklearn.metrics import balanced_accuracy_score
print('Balance Accuracy Score modelo DummyClassifier : ', balanced_accuracy_score(y_test, y_pred))

Balance Accuracy Score modelo DummyClassifier :  0.31058535785532754


```
Podemos ver claramente que el modelo dummy se comporta peor en comparacion a los resutlados del modelo baseline, por ejemplo cuando analizamos la metrica de precision, que indica el número de verdaderos positivos que son realmente positivos en comparación con el número total de valores positivos predichos, podemos ver que claramente que para predecir los malos y buenos personajes el primer modelo es el mejor, ademas cuando vemos el accuracy normal tambien podemos ver que le resultado del primer modelo es mejor y es mas, cuando usamos balanced accuracy para tener en cuenta el desbalance de los datos el primer modelo se sigue comportando de mejor manera, lo cual nos indica que al menos esta clasificando mejor que al azar.


```

## 1.3 Busqueda del Mejor Modelo con Grid Search [4 Puntos]

<p align="center">
  <img src="https://media1.tenor.com/images/70fdfeea52a8e2e4505498c230a0d2f9/tenor.gif?itemid=5134219" width="250">
</p>

No conformes con el rendimiento obtenido en la sección 1.2, el cuerpo docente les pide que realicen un **`HalvingGridSearchCV`** con diferentes parámetros para mejorar el rendimiento de la clasificación. Para esto, se le solicita que defina:

- Tres clasificadores distintos en donde varie sus parámetros. Considere usar modelos clásicos como también los basados en ensamblaje.
- Modificar `n-gram` range del `CountVectorizer` probando `(1,1), (1,2) y (1,3)`. Examinar también los otros parámetros de CountVectorizer como por ejemplo `max_df`, `min_df`, etc... ([Documentación aquí](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html))
- Seleccionar las columnas que contribuyen con la mayor información para la clasificación con `SelectPercentile` en los percentiles `[20, 40, 60, 80]` (puede usar la métrica que usted quiera).
- Reporte la mejor combinación encontrada y justifique por qué cree que es la mejor según el clasificador usado, la cantidad de columnas seleccionadas y los parámetros de CountVectorizer seleccionados por GridSearch.

A continuación, un ejemplo de parametros para GridSearch para una búsqueda de 3 clasificadores distintos:

```python
params = [
       # clasificador 1 + hiperparámetros
       {'clf': classificator1(),
        'clf__penalty': ['ovr'],
       # clasificador 1 + hiperparámetros    
       {'clf': classificator2(),
        'clf__n_estimators': [200]},
       # clasificador 1 + hiperparámetros
       {'clf': classificator3(),
        ...
       }
       ]
```

**Nota 1**: Puede ver los parámetros modificables aplicando el método get_params() sobre su pipeline. Ver la clase de GridSearch para mayor información sobre la sintáxis de las grillas.

**Nota 2**: Recuerde inicializar los clasificadores con un random state definido.

**Nota 3**: Puede usar en `HalvingGridSearchCV` el parámetro `verbose=10` para ver que GridSearch le indique el estado de su ejecución.

**Nota 3:** El GridSearch puede tomar tiempos de búsqueda exorbitantes, por lo que se le recomienda no agrandar mucho el espacio de búsqueda, dejar corriendo el código y tomarse un tecito.

**Respuesta:**

In [22]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.experimental import enable_halving_search_cv  
from sklearn.model_selection import HalvingGridSearchCV


params_mb = [{
    'Preprocessing__Texto__ngram_range' : [(1, 1), (1, 2), (1, 3)],
    'Selection__percentile' : [20, 40, 60, 80],
    'Clasificador__alpha' : [0.5, 1],
    'Clasificador__fit_prior' : [True, False]
}]

param_rf = [{
    'Preprocessing__Texto__ngram_range' : [(1, 1), (1, 2), (1, 3)],
    'Selection__percentile' : [20, 40, 60, 80],
    'Clasificador__n_estimators' : [80, 100, 120],
    'Clasificador__criterion' : ['gini', 'entropy']
}]

param_kn = [{
    'Preprocessing__Texto__ngram_range' : [(1, 1), (1, 2), (1, 3)],
    'Selection__percentile' : [20, 40, 60, 80],
    'Clasificador__n_neighbors': [2, 4, 5, 10]
}]



In [23]:
model_mb = Pipeline(steps=[
                           ('Preprocessing', preprocessor),
                           ("Selection", SelectPercentile(f_classif, percentile=90)),
                           ('Clasificador', MultinomialNB())
])

model_rf = Pipeline(steps=[
                           ('Preprocessing', preprocessor),
                           ("Selection", SelectPercentile(f_classif, percentile=90)),
                           ('Clasificador', RandomForestClassifier())
])

model_kn = Pipeline(steps=[
                           ('Preprocessing', preprocessor),
                           ("Selection", SelectPercentile(f_classif, percentile=90)),
                           ('Clasificador', KNeighborsClassifier())
])


In [24]:
model_kn.get_params()

{'memory': None,
 'steps': [('Preprocessing', ColumnTransformer(transformers=[('Texto',
                                    CountVectorizer(ngram_range=(1, 2),
                                                    tokenizer=<__main__.StemmerTokenizer object at 0x7fd5bcf2e110>),
                                    'history_text'),
                                   ('Numeric', MinMaxScaler(),
                                    ['intelligence_score', 'strength_score',
                                     'speed_score', 'durability_score',
                                     'power_score', 'combat_score'])])),
  ('Selection', SelectPercentile(percentile=90)),
  ('Clasificador', KNeighborsClassifier())],
 'verbose': False,
 'Preprocessing': ColumnTransformer(transformers=[('Texto',
                                  CountVectorizer(ngram_range=(1, 2),
                                                  tokenizer=<__main__.StemmerTokenizer object at 0x7fd5bcf2e110>),
                          

In [25]:
model_mb = HalvingGridSearchCV(model_mb, params_mb, random_state = 0, n_jobs=-1).fit(X_train,y_train)

In [26]:
y_pred_mb = model_mb.predict(X_test)

In [27]:

#Comparamos con classification_report
print('Classification report')
print(classification_report(y_test, y_pred_mb))

Classification report
              precision    recall  f1-score   support

         Bad       0.00      0.00      0.00        86
        Good       0.57      0.99      0.72       148
     Neutral       0.00      0.00      0.00        23

    accuracy                           0.57       257
   macro avg       0.19      0.33      0.24       257
weighted avg       0.33      0.57      0.42       257



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [28]:
model_rf = HalvingGridSearchCV(model_rf, param_rf, random_state = 0, n_jobs=-1).fit(X_train,y_train)

In [29]:
y_pred_rf = model_rf.predict(X_test)

In [30]:

#Comparamos con classification_report
print('Classification report')
print(classification_report(y_test, y_pred_rf))

Classification report
              precision    recall  f1-score   support

         Bad       0.68      0.22      0.33        86
        Good       0.62      0.95      0.75       148
     Neutral       0.50      0.04      0.08        23

    accuracy                           0.62       257
   macro avg       0.60      0.40      0.39       257
weighted avg       0.63      0.62      0.55       257



In [31]:
model_kn = HalvingGridSearchCV(model_kn, param_kn, random_state = 0, n_jobs=-1).fit(X_train,y_train)

In [35]:
y_pred_kn = model_kn.predict(X_test)

In [36]:
#Comparamos con classification_report
print('Classification report')
print(classification_report(y_test, y_pred_kn))

Classification report
              precision    recall  f1-score   support

         Bad       0.33      0.01      0.02        86
        Good       0.58      0.99      0.73       148
     Neutral       0.00      0.00      0.00        23

    accuracy                           0.58       257
   macro avg       0.30      0.33      0.25       257
weighted avg       0.44      0.58      0.43       257



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [37]:
model_rf.best_estimator_

Pipeline(steps=[('Preprocessing',
                 ColumnTransformer(transformers=[('Texto',
                                                  CountVectorizer(tokenizer=<__main__.StemmerTokenizer object at 0x7fd5b974b110>),
                                                  'history_text'),
                                                 ('Numeric', MinMaxScaler(),
                                                  ['intelligence_score',
                                                   'strength_score',
                                                   'speed_score',
                                                   'durability_score',
                                                   'power_score',
                                                   'combat_score'])])),
                ('Selection', SelectPercentile(percentile=60)),
                ('Clasificador',
                 RandomForestClassifier(criterion='entropy',
                                        n_estimators=120)

```
Tal como ser puede ver en la metrica precision Random Forest obtuvo el mejor resultado, tanto para Bad, Good y Neutral, sumado a esto, se puede apreciar que cuando se compara el accuracy se puede ver que Random Forest siendo el mejor, alcanzando un accuracy de 0.62, además se puede ver que las columnas seleciconadas fueron: 'intelligence_score','strength_score','speed_score', 'durability_score', 'power_score' y 'combat_score', ademas la mejor combinacion de los hiperparametros fue criterion = 'entropy' y n_estimators = 120

```

## 1.4 Predicción del datos sin etiquetado  [0.5 puntos]

<p align="center">
  <img src="https://pbs.twimg.com/media/DolotxUUYAAbg7f.jpg" width="350">
</p>


LLego el momento de predecir 
`Vergil`, `Gorilla Girl` y `Batcow`


**Nota:** Recuerde que pueden existir campos vacios en `history_text`, por lo que se les recomienda borrar los nan.

**Respuesta:**

In [56]:
#tal como se indica en la nota se borran los nan de history_text
df_comics_no_label = df_comics_no_label.dropna(subset=['history_text']) 
df_comics_no_label = df_comics_no_label.drop_duplicates(subset = ['name'])

X = df_comics_no_label[atributos]

#Se predice con Random Fores, porque como se vio en la parte anterior fue el mejor modelo
y_pred = model_rf.predict(X)

#Se crea una columna con los valores predichos
df_comics_no_label['alignment'] = y_pred

#Veamos como predice nuestro modelo Vergil, Gorilla Girl y Batcow

print('Vergil es predicho como ', str(df_comics_no_label[df_comics_no_label['name'] == 'Vergil']['alignment'].values))
print('Gorilla Girl es predicha como ', df_comics_no_label[df_comics_no_label['name'] == 'Gorilla Girl']['alignment'].values)
print('Batcow es predicha como ', df_comics_no_label[df_comics_no_label['name'] == 'Batcow']['alignment'].values)

Vergil es predicho como  ['Good']
Gorilla Girl es predicha como  ['Good']
Batcow es predicha como  ['Good']


<p align="center">
  <img src="https://media1.tenor.com/images/fb5bf7cc5a4acb91b4177672886a88ba/tenor.gif?itemid=5591338">
</p>

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=87110296-876e-426f-b91d-aaf681223468' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>